In [1]:
import pandas as pd

In [4]:
df = pd.read_excel('database.xlsx', sheet_name='Raw', header=0)

# Define a function to convert a cell value into a list of integers
def cell_to_list(cell_value):
    if isinstance(cell_value, str):
        # Split and convert to integers if it's a string
        return [int(i) for i in cell_value.split(', ')]
    elif pd.notna(cell_value):
        # If it's not a string but not empty, consider it as an integer
        return [int(cell_value)]
    else:
        # Handle empty cells or NaN values
        return []

# Apply the function to the 'UID PM' column
df['UID PM'] = df['UID PM'].apply(cell_to_list)
df['UID EM'] = df['UID EM'].apply(cell_to_list)
df['UID GRI'] = df['UID GRI'].apply(cell_to_list)

all_uid = df['UID'].tolist()

In [5]:
#Check reciprocity of the relationship
for i, row in df.iterrows():
    pm = []
    em = []
    if row['UID PM'] != []:
        pm = row['UID PM'].copy()
    if row['UID EM'] != []:
        em = row['UID EM'].copy()
    if row['UID GRI'] != []:
        if row['Match Type GRI'] == 'em':
            em = em + row['UID GRI']
        elif row['Match Type GRI'] == 'pm':
            pm = pm + row['UID GRI']
        else:
            # separate the elements of uid gri and match type gri
            gri = row['UID GRI']
            row['Match Type GRI'] = str(row['Match Type GRI'])
            gri_type = row['Match Type GRI'].split(', ')
            for i in range(len(gri)):
                if gri_type[i] == 'em':
                    em.append(gri[i])
                elif gri_type[i] == 'pm':
                    pm.append(gri[i])
                else:
                    print('Error: match type not recognized', row['UID'], row['Match Type GRI'], row['UID GRI'])
    if pm != []:
        for uid in pm:
            if uid in all_uid:
                if df[df['UID'] == uid]['UID PM'].values[0] != []:
                    # check if row['UID'] is in the list of 'UID PM' of the row with uid
                    if row['UID'] not in df[df['UID'] == uid]['UID PM'].values[0]:
                        # if not, add it
                        df[df['UID'] == uid]['UID PM'].values[0].append(row['UID'])
            else:
                row['UID PM'] = row['UID PM'].remove(uid)
    if em != []:
        for uid in em:
            if uid in all_uid:
                if df[df['UID'] == uid]['UID EM'].values[0] != []:
                    # check if row['UID'] is in the list of 'UID EM' of the row with uid
                    if row['UID'] not in df[df['UID'] == uid]['UID EM'].values[0]:
                        # if not, add it
                        df[df['UID'] == uid]['UID EM'].values[0].append(row['UID'])
            else:
                row['UID EM'] = row['UID EM'].remove(uid)

In [ ]:
# find min max of column 'Framework' == GRI
minGRI = df.loc[df['Framework'] == 'GRI', 'UID'].min()
maxGRI = df.loc[df['Framework'] == 'GRI', 'UID'].max()

# Move GRI metris to GRI column
for i, row in df.iterrows():
    if row['UID PM'] != []:
        for x in row['UID PM']:
            if x >= minGRI and x <= maxGRI:
                if x not in row['UID GRI']:
                    row['UID GRI'].append(x)
                row['UID PM'].remove(x)
    if row['UID EM'] != []:
        for x in row['UID EM']:
            if x >= minGRI and x <= maxGRI:
                if x not in row['UID GRI']:
                    row['UID GRI'].append(x)
                row['UID EM'].remove(x)

In [ ]:
# For each row, if the UID in the UID PM, then delte that UID from the UID PM, UID EM, UID GRI
for i, row in df.iterrows():
    if row['UID'] in row['UID PM']:
        row['UID PM'] = row['UID PM'].remove(row['UID'])
    if row['UID'] in row['UID EM']:
        row['UID EM'] = row['UID EM'].remove(row['UID'])
    if row['UID'] in row['UID GRI']:
        row['UID GRI'] = row['UID GRI'].remove(row['UID'])

# For each row, if the UID EM in UID PM, then delte that UID EMfrom the UID PM
for i, row in df.iterrows():
    if row['UID PM'] != []:
        if row['UID EM'] != []:
            for x in row['UID EM']:
                if x in row['UID PM']:
                    row['UID PM'].remove(x)

In [ ]:
df.to_excel('database_clean.xlsx', sheet_name='Raw', index=False)

In [296]:
print(df.head())

   UID  NUID                         Name  \
0  210     1   Age Diversity in Workforce   
1  211     2     Measurement of Diversity   
2  212     3   Inclusive Hiring Practices   
3  213     4  Inclusive Work Environments   
4  214     5           Low Income Workers   

                                         Description  \
0  What percentage of your workforce is either un...   
1  What attributes of a diverse workforce does yo...   
2  How does your company create an inclusive recr...   
3  How does your company create an equitable and ...   
4  What percentage of your workforce lives in poo...   

                            Topic       Metric scope          Type  \
0                  diversity: age  direct operations  Quantitative   
1  diversity: diverse measurement  direct operations   Qualitative   
2                inclusive hiring  direct operations   Qualitative   
3            diversity management  direct operations   Qualitative   
4                   living income  direct 

In [293]:
# Create a map were we change all UID numbers by the number in NUID
uid_map = {}
for i, row in df.iterrows():
    uid_map[row['UID']] = row['NUID']

# Change the UIDs in the UID PM, UID EM, UID GRI columns
for i, row in df.iterrows():
    if row['UID PM'] != []:
        for x in range(len(row['UID PM'])):
            row['UID PM'][x] = uid_map[row['UID PM'][x]]
    if row['UID EM'] != []:
        for x in range(len(row['UID EM'])):
            row['UID EM'][x] = uid_map[row['UID EM'][x]]
    if row['UID GRI'] != []:
        for x in range(len(row['UID GRI'])):
            row['UID GRI'][x] = uid_map[row['UID GRI'][x]]